In [1]:
import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#set up Mongo to store data 
client = MongoClient('localhost', 27017)

db = client['cap1_db']
col1 = db['col1']
coltest = db['coltest']
name_and_link_col = db['name_and_link']


In [3]:
category_url = ['Data_scientists', 
            '21st-century_economists', 
            '21st-century_chemists', 
            '21st-century_physicists',
           '21st-century_biologists']

subcategory_url = set()

In [4]:
def html_to_collection(collection, page):
    
    doc = collection.find_one({'page': page })    
    try:
        doc['page']
        print(f"{page} page already exists!")
    
    except:
        wik = 'https://en.wikipedia.org/wiki/Category:'
        url = wik + page
        r = requests.get(url)
        collection.insert_one({'page': page, 
                               'html':r.content})
        print(f"{page}: I saved the html")
        
    return None 


def people_to_collection(collection, page):
    
    doc = collection.find_one({'page': page })
    
    try:
        doc['list_names']
        print(f"{page} names already here!")
    
    except:
        try:

            soup = BeautifulSoup(doc['html'] ,'html.parser')
            div = soup.find("div", {'class':'mw-category'})

            list_link_ext = []
            for link in div.find_all('a'):
                list_link_ext.append(link['href'])
            print(f"{page}:i grabbed a person link!")

            #grab names
            list_names = []
            for link in div.find_all('a'):
                list_names.append(link['title'])
            print(f"{page}:i grabbed a name!")

            collection.update_one({'page': page },{"$set":{'list_names': list_names,
                                                          'list_name_links': list_link_ext
                                                         }})
        except: 
            print(f"{page} oh no i couldn't find people")
        return None


def find_subcategories(collection, page):
    
    doc = collection.find_one({'page': page })
    
    try:
        doc['subcat_name']
        print(f"{page} subcategories already here!")
    
    except:
        try:
            soup = BeautifulSoup(doc['html'] ,'html.parser')

            div = soup.find("div", {'id':'mw-subcategories'})

            subcat_link = []
            for link in div.find_all('a'):
                subcat_link.append(link['href'])
            print(f"{page}: i added a subcategory link")

            subcat_name = []
            for link in div.find_all('a'):
                subcat_name.append(link['title'])
            print(f"{page}: i added a subcategory name")

            collection.update_one({'page': page },{"$set":{'subcat_name': subcat_name,
                                                          'subcat_link': subcat_link
                                                         }})
        except:
            print(f"{page}: I have no subcategories")
        return None
        


def find_next_page(collection, page):
    
    doc = collection.find_one({'page': page })
    
    try:
        doc['next_page']
        print(f"{page}: next page already here!")
    
    except:
        try:
            soup = BeautifulSoup(doc['html'] ,'html.parser')
            div = soup.find("div", {'id':'mw-pages'})   

            links = []
            for link in div.find_all('a'):
                links.append(link['href'])

            last_link = links.pop()
            if 'pagefrom' in last_link:
                collection.update_one({'page': page },{"$set":{'next_page': last_link}})
                print(f"{page}:I grabbed the next page")
            else:
                print(f"{page}:I'm already the last page")    
        
        except:
            collection.update_one({'page': page },{"$set":{'next_page': 'only page'}})
            print(f"{page}: I'm the only page")
    
    return None


def subcat_html_to_collection(collection, page):
    
    doc = collection.find_one({'page': page })
    
    try: 
        subcat_link = doc['subcat_link']
        wik = 'https://en.wikipedia.org/'

        for i in subcat_link:
            url = wik + i
            r = requests.get(url)
            subcat_page = i.replace('/wiki/Category:', '')

            if subcat_page in subcategory_url:
                print(f"{subcat_page}: subcategory html already here")
            elif subcat_page not in subcategory_url:
                subcategory_url.add(subcat_page)
                collection.insert_one({'page': subcat_page, 
                               'html':r.content})
                print(f"{subcat_page}: I saved the subcat html")

    except:
        print(f"{page} I don't have subcategories or they are already added")



In [5]:
for i in category_url:
    
    html_to_collection(col1, i)
    people_to_collection(col1, i)
    find_subcategories(col1, i)
    find_next_page(col1, i)
    subcat_html_to_collection(col1, i)


Data_scientists: I saved the html
Data_scientists:i grabbed a person link!
Data_scientists:i grabbed a name!
Data_scientists: i added a subcategory link
Data_scientists: i added a subcategory name
Data_scientists:I'm already the last page
Women_data_scientists: I saved the subcat html
21st-century_economists: I saved the html
21st-century_economists:i grabbed a person link!
21st-century_economists:i grabbed a name!
21st-century_economists: i added a subcategory link
21st-century_economists: i added a subcategory name
21st-century_economists:I grabbed the next page
21st-century_American_economists: I saved the subcat html
21st-century_Dutch_economists: I saved the subcat html
Milton_Friedman: I saved the subcat html
21st-century_Indian_economists: I saved the subcat html
21st-century_chemists: I saved the html
21st-century_chemists:i grabbed a person link!
21st-century_chemists:i grabbed a name!
21st-century_chemists: i added a subcategory link
21st-century_chemists: i added a subcatego

In [6]:
for i in subcategory_url:
    people_to_collection(col1, i)
    find_subcategories(col1, i)
    find_next_page(col1, i)


21st-century_American_biologists:i grabbed a person link!
21st-century_American_biologists:i grabbed a name!
21st-century_American_biologists: i added a subcategory link
21st-century_American_biologists: i added a subcategory name
21st-century_American_biologists:I grabbed the next page
21st-century_British_biologists:i grabbed a person link!
21st-century_British_biologists:i grabbed a name!
21st-century_British_biologists: i added a subcategory link
21st-century_British_biologists: i added a subcategory name
21st-century_British_biologists:I'm already the last page
21st-century_zoologists:i grabbed a person link!
21st-century_zoologists:i grabbed a name!
21st-century_zoologists: i added a subcategory link
21st-century_zoologists: i added a subcategory name
21st-century_zoologists:I'm already the last page
Women_data_scientists:i grabbed a person link!
Women_data_scientists:i grabbed a name!
Women_data_scientists: I have no subcategories
Women_data_scientists:I'm already the last page


In [7]:
doc = col1.find_one({'page':'21st-century_American_economists'})

In [11]:
doc['next_page']

'/w/index.php?title=Category:21st-century_American_economists&pagefrom=Constantinides%2C+George%0AGeorge+Constantinides#mw-pages'